# **JOINEMIO**
## A project that uses AI to play Connect4
Project created by three AGH students:
* <a href="https://github.com/shoomilas" target="_blank">Jakub Szumilas</a>
* <a href="https://github.com/KarolSzeliga4" target="_blank">Karol Szeliga</a>
* <a href="https://github.com/gzelek8" target="_blank">Grzegorz Zelek</a>

Final project for the `PitE-Summer-2021` subject. Its main goal was to train the AI ​​player in such a way that he could smoothly play the game against a player making random moves. 
The implementation was divided into several stages: 
* 1.The first was to create an imitation of a game with a graphical interface where it was possible to play two players. 
* 2.The next step was to create the game environment needed to implement the next steps. An open ai gym environment was used for this traffic jam. 
* 3.Then, an appropriate method and an appropriate algorithm were selected to solve the problem. The choice fell on alfoytm deep q learning and the use of the Multi-armed bandit alogytm. 
* 4.The last step was to select the appropriate parameters and summarize the results. 

The project achieved its intended goal and after training most of the games, it beats the opponent making random moves.

## Initial imports
### The libraries used are:
* `pytorch` (assistance in the implementation of the neural network and the network learning process)
* `numpy` (assistance with operations related to the board)
* `gym` (assistance with the implementation of the environment)
* `neptune` (assistance in monitoring the learning process)


In [ ]:
import neptune
import neptune.new as neptune
import os

token = os.getenv('NEPTUNE_API_TOKEN')
proj = 'joinemio/test'
run = neptune.init(project=proj,
                   api_token=token)
import time
import gym
import torch
import random
from collections import defaultdict, deque
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from collections import namedtuple, deque

import numpy as np
import copy
from pprint import pprint
import math
from tqdm.autonotebook import tqdm
from torch.utils.data import DataLoader
from torchvision import transforms
from joblib import Parallel, delayed
import gc
import gym_joinemio
from gym_joinemio.envs.player import RandomPlayer
from gym_joinemio.envs.connect_four_env import Reward

We made an environment implanted by ourselves, which helped us understand the entire preparation process.

In [ ]:
env = gym.make('joinemio-v0')

In order to speed up the learning process, we used calculations with the use of a graphics card

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

## Replay memory
All of the agent's experiences at each time step over all episodes played by the agent are stored in the replay memory. Well actually, in practice, we'll usually see the replay memory set to some finite size limit, , and therefore, it will only store the last  experiences.

This replay memory data set is what we'll randomly sample from to train the network. The act of gaining experience and sampling from the replay memory that stores these experience is called experience replay.

A key reason for using replay memory is to break the correlation between consecutive samples.
If the network learned only from consecutive samples of experience as they occurred sequentially in the environment, the samples would be highly correlated and would therefore lead to inefficient learning. Taking random samples from replay memory breaks this correlation.
***
### Replay memory steps:
 - Initialize replay memory capacity.
 - Initialize the network with random weights.
 - For each episode:
 
   - Initialize the starting state.

   - For each time step:

      - Select an action.

        - Via exploration or exploitation

      - Execute selected action in an emulator.

      - Observe reward and next state.
            
      - Store experience in replay memory.
***

In [ ]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward'))

class ReplayMemory(object):
    def __init__(self, capacity):
        self.memory = deque([],maxlen=capacity)

    def push(self, *args):
        """Save a transition"""
        self.memory.append(Transition(*args))

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size) # TODO: Batch size delete

    def __len__(self):
        return len(self.memory)

## Training parameters: 
Random vs Random. Deep Q-Learning. Params:

- `n_episodes` (int): maximum number of training epsiodes
- `max_t` (int): maximum number of timesteps per episode _// Not used, because these episodes don't take too long and we like when game's are finished_
- `eps_start` (float): starting value of epsilon, for epsilon-greedy action selection
- `eps_end` (float): minimum value of epsilon 
- `eps_decay` (float): mutiplicative factor (per episode) for decreasing epsilon

In [ ]:
params = {
	"EPISODES" 					: 10,
	"EPS_START" 				: 1.0,
	"EPS_END" 					: 0.01,
	"LEARNING_RATE" 			: 1e-4,
	"EPS_DECAY" 				: 0.996,
	"EPS_DECAY_LAST_FRAME" 		: 10**5,
	"BUFFER_SIZE"				: 300,
	"FRAMES_MIN" 				: 15000,
	"MEAN_REWARD_BOUND" 		: 0.80,
	"SYNC_TARGET_FRAMES" 		: 1000,
	"REPLAY_START_SIZE" 		: 10000
}

run["parameters"] = params

EPISODES = params["EPISODES"]
EPS_START = params["EPS_START"]
EPS_END = params["EPS_END"]
LEARNING_RATE = params["LEARNING_RATE"]
EPS_DECAY = params["EPS_DECAY"]
EPS_DECAY_LAST_FRAME = params["EPS_DECAY_LAST_FRAME"] # 10**5

BUFFER_SIZE = params["BUFFER_SIZE"]
FRAMES_MIN = params["FRAMES_MIN"]
MEAN_REWARD_BOUND = params["MEAN_REWARD_BOUND"]
N_FOR_MEAN_REWARD = BUFFER_SIZE

SYNC_TARGET_FRAMES = params["SYNC_TARGET_FRAMES"]
REPLAY_START_SIZE = params["REPLAY_START_SIZE"]

## NeuralNetwork
For the learning process, we used a neural network consisting of 4 layers, 42 input neurons on each of them and  7 neurons, each corresponding to the action of dropping a chip into one of 7 different columns of the board. At the input, we place a flattened board with 42 elements depicting the game boards, at the output the profitability weight of the token toss on a given selected column

In a neural network, the activation function is responsible for transforming the summed weighted input from the node into the activation of the node or output for that input. The rectified linear activation function or ReLU for short is a piecewise linear function that will output the input directly if it is positive, otherwise, it will output zero. It has become the default activation function for many types of neural networks because a model that uses it is easier to train and often achieves better performance.

We used `relu` functions as activation functions on our layers
This function returns 0 if it receives any negative input, but for any positive value  x  it returns that value back. So it can be written as  f(x)=max(0,x)

***
*Neural Network Architecture Diagram.*

![Neural Network Architecture Scheme.](https://raw.githubusercontent.com/shoomilas/joinemio/main/docs/img/NeurNetArch.png)
***

In [ ]:
from gym_joinemio.envs.board import Board, GameState


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.main_layers = nn.Sequential(
            nn.Linear(6*7, 6*7),
            nn.ReLU(),
            nn.Linear(6*7, 6*7),
            nn.ReLU(),
            nn.Linear(6*7, 6*7),
            nn.ReLU()
        )
        self.output_layer = nn.Linear(6*7,7)

    def forward(self, board_flatten_state):
        weights = self.output_layer(self.main_layers(board_flatten_state))
        return weights

## AIPlayer


The learning process was carried out using deep q learning. ore specifically, the agents receives information on the current observation (the current state of the board) and then has to take an action (which slot to choose to add a coin). After that, nature responses with a new state and potentially yields a reward (if the game is won) or a penalty (if the game is lost or if the agent chooses an action that is not valid - such as putting a coin into an already full slot). The goal of each action is to receive the greatest possible reward. After gathering some experience, a neural network is trained to make sense of the state, action and reward relationship. The target is set such that the network aims at minimizing the loss between predicting the reward of the next_state and the realized reward.

In all `Reinforcement Learning` problems, there exists an unavoidable trade-off between exploration and exploitation. Exploration is where an agent makes some out of character decision (not dictated by the network) to try to find new strategies and potentially be rewarded from them; thus, an opportunity to learn better strategies. Exploitation is to exploit the strategies the network has already learned. For example, if an agent has learned a bit how to play Connect 4 and it always exploits the techniques it has already learned, it will never be presented with new observation/action pairs to learn from. On the other hand, if the agent is always exploring new strategies, it is essentially always playing random moves and never reaching a high level of game play to learn from. The way this problem is dealt with typically is through a phenomenon known as epsilon decay. The epsilon in epsilon decay can be thought of as the probability the agent will choose a random action rather than an action as chosen by then network. The decay in epsilon decay is just that: since towards the beginning of the training cycle we expect the agent to be dumb, for lack of a better term, we will let it make largely random decisions, thus letting the agent explore more than exploit, since it hasn’t yet learned anything to exploit. Over time, the epsilon will get smaller and smaller, that is: the agent will start exploiting its learned knowledge more and more; and will explore random actions less and less. 

In [ ]:
class AIPlayer:
    @staticmethod
    def possible_moves(board_state):
        available_cols = []
        for i in range(len(board_state[0])):
            if board_state[0][i] == 0:
                available_cols.append(i)
        return available_cols

    def __init__(self, env, replay_memory, net):
        self.net = net
        self.env = env
        self.env.opponent_action_set(RandomPlayer.get_action)
        self.replay_memory = replay_memory
        self._reset()

    def _reset(self):
        self.env = gym.make('joinemio-v0')
        self.state =  self.env.reset()
        self.env.opponent_action_set(RandomPlayer.get_action)
        self.env.game = gym_joinemio.envs.board.Game()
        self.total_reward = 0.0

    def get_action(self, board_state):
        weigths = self.net.forward(torch.flatten(torch.FloatTensor(board_state.astype(float)).type(torch.FloatTensor)))
        pos_nums = self.possible_moves(board_state)
        max_num = 0
        for col in pos_nums:
            if weigths[max_num] < weigths[int(col)]:
                max_num = int(col)
        return max_num

    def get_net_action(self, grid):
        state_a = np.array(self.state, copy=False, dtype=np.uint8)
        state_v = torch.flatten(torch.FloatTensor(state_a).to(device))
        q_vals_v = self.net(state_v)
        # TODO filter
        for i in range(Board.columns):
            if i not in self.possible_moves(grid):
                q_vals_v[i] = -100
        _, act_v = torch.max(q_vals_v, dim=0)
        action = int(act_v.item())
        return action

    def play_step(self, net, epsilon, device):
        done_reward = None
        final_reward = None
        ## print("*AI PLAYER PLAYED*")
        state_a = np.array(self.state, copy=False, dtype=np.uint8)
        state_v = torch.flatten(torch.FloatTensor(state_a).to(device))
        q_vals_v = net(state_v)

        grid = self.env.game.board.grid
        for i in range(Board.columns):
            if i not in self.possible_moves(grid):
                q_vals_v[i] = -1
        _, act_v = torch.max(q_vals_v, dim=0)
        action = int(act_v.item())  

        new_state, reward, is_done, _ = self.env.step(action)
        final_reward = reward
        self.total_reward += reward.value

        self.replay_memory.push(self.state, action, new_state, reward)
        self.state = new_state

        if is_done == GameState.finished:
            done_reward = self.total_reward
            self._reset()
            return (done_reward, final_reward)
        else: return (done_reward, None)

    def train(self, memory_buffer, batch_size):
        return 0


## Training loop:
Training is nothing as iteratively playing against the trainer, memorizing what happened and updating the neural net weights after each iteration.

So it looks like `Deep-Q-Learning` was the right choice: just by playing against a random agent, the neural network was trained to win the game - even without knowing the rules first!

***
*Deep Q Learning - An example of a path to a game-ending state in a game subtree*

![DEEP Q.](https://miro.medium.com/max/700/1*NKzsRiAxa_oiikgbLyLCyw.png)
***

In [ ]:
from gym_joinemio.envs.game_window import GameWindow
from pyglet import clock
from pyglet import app

def play_with_ai(agent):
    window = GameWindow(agent)
    clock.schedule_interval(window.update, 1000) 
    app.run()

def our_main():
    env = gym.make('joinemio-v0')
    net = NeuralNetwork().to(device)
    tgt_net = NeuralNetwork().to(device)
    print(net)
    buffer = ReplayMemory(BUFFER_SIZE)
    global agent 
    agent = AIPlayer(env, buffer, net)
    epsilon = EPS_START
    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE)
    
    total_rewards = []
    frame_idx = 0
    ts_frame = 0
    ts = time.time()
    best_mean_reward = None
    last_total_frames = 0
    while True:
        frame_idx += 1
        epsilon = max(EPS_END, EPS_START - frame_idx / EPS_DECAY_LAST_FRAME)
        reward, final_reward = agent.play_step(net, epsilon, device=device)
        if (final_reward is not None):
            frames_this_game = frame_idx - last_total_frames
            last_total_frames = frame_idx
            total_rewards.append(reward)
            ts_frame = frame_idx
            ts = time.time()
            mean_reward = np.mean(total_rewards[-N_FOR_MEAN_REWARD:])
            run["eps"].log(epsilon)
            run["games_done"].log(len(total_rewards))
            run["mean_reward"].log(mean_reward)
            
            if best_mean_reward is None or best_mean_reward < mean_reward:
                torch.save(net.state_dict(), "joinemio-best.dat") # TODO: Extract
                if best_mean_reward is not None:
                    print(f"Best mean reward updated {best_mean_reward}->{mean_reward}; model saved")
                best_mean_reward = mean_reward

            condition = mean_reward > MEAN_REWARD_BOUND and frame_idx > FRAMES_MIN
            if condition:
                break
        else: pass 
        
        if len(buffer) < REPLAY_START_SIZE:
            continue

        if frame_idx % SYNC_TARGET_FRAMES == 0:
            tgt_net.load_state_dict(net.state_dict())

        optimizer.zero_grad()
        batch = buffer.sample(BATCH_SIZE)
        optimizer.step()

our_main()

In [ ]:
run.stop()

In [ ]:
play_with_ai(agent)

## Conclusions and observations
We have brought the network to the stage where the network learns when playing with a player making random moves. We had a problem with looping the learning process, so we used multi-armed bandit algorithm. It allowed us to break these loops. Initially, the learning player performs random moves in the same way as the opponent, but with subsequent moves, moves selected by the network are interfered more and more often. The parameter ε is responsible for the frequency of these movements. An increasingly well-trained network makes more and more non-random moves, which leads to a much larger number of won games. The network discovered that when playing with a player making random moves, the best chance of winning is by placing tokens in one column all the time.

Deep Q-Learning can handle any observation space so long as the observations themselves are not too data intensive for the network. I feel this sort of algorithm is more useful to experiment with as it is more likely to be implemented in real world environments